In [15]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score

In [30]:
df = pd.read_csv('bank-additional-full.csv', delimiter=';')
df = df.replace(['unknown'], np.NaN)
df = df.dropna()
df['y'] = df['y'].map({'yes': 1, 'no': 0})

X = df.drop(['y'], axis=1)
y = df['y']

In [31]:
cat = list(X.select_dtypes(['object']).columns)
cont = list(X.select_dtypes(['int64']).columns)

cont_transform = Pipeline(steps=[('scaler', StandardScaler())])

cat_transform = Pipeline(steps=[('categories', OneHotEncoder(sparse=False, handle_unknown='ignore'))])

preprocessor = ColumnTransformer(transformers=[('cont', cont_transform, cont),
                                               ('cat', cat_transform, cat)])
X = pd.DataFrame(preprocessor.fit_transform(X))
X

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,51
0,1.642253,0.005792,-0.559335,0.211887,-0.371616,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,-0.196452,-0.127944,-0.559335,0.211887,-0.371616,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.093870,-0.414520,-0.559335,0.211887,-0.371616,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.642253,0.181559,-0.559335,0.211887,-0.371616,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1.932575,-0.460373,-0.559335,0.211887,-0.371616,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30483,3.287410,0.284727,-0.559335,0.211887,-0.371616,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
30484,0.674513,0.471957,-0.559335,0.211887,-0.371616,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
30485,1.642253,-0.269321,-0.191702,0.211887,-0.371616,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
30486,0.480965,0.697398,-0.559335,0.211887,-0.371616,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [39]:
# Logistic Regression
pipe = Pipeline(steps=[('classifier', LogisticRegression())])

parameters = [{'classifier': [LogisticRegression(max_iter=5000)],
               'classifier__solver': ['saga'],
               'classifier__penalty': ['l1'],
               'classifier__C': np.logspace(-8,4,13)},
              {'classifier': [LogisticRegression(max_iter=5000)],
               'classifier__solver': ['sag', 'saga'],
               'classifier__penalty': ['none']},
              {'classifier': [LogisticRegression(max_iter=5000)],
               'classifier__solver': ['sag', 'saga'],
               'classifier__penalty': ['l2'],
               'classifier__C': np.logspace(-8,4,13)}]

clf = GridSearchCV(pipe, parameters, cv=KFold(n_splits=5, shuffle=True), 
                   scoring=['accuracy', 'precision', 'roc_auc', 'f1'], refit=False)

In [40]:
# change num_trials = 5
num_trials = 1
train_metrics = pd.DataFrame(columns=['Accuracy','Precision','AUC','F1'])
for i in range(num_trials):
    # change train size to 5000
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=500, shuffle=True)
    hyperparams = clf.fit(X_train, y_train)
    results = hyperparams.cv_results_['params']
    solution = pd.DataFrame(results)
    
    # ACCURACY
    solution['Accuracy'] = hyperparams.cv_results_['mean_test_accuracy']
    best_accuracy = results[np.argmin(hyperparams.cv_results_['rank_test_accuracy'])]
    accuracy_model = LogisticRegression(penalty = best_accuracy['classifier__penalty'],
                                        C = best_accuracy['classifier__C'],
                                        solver = best_accuracy['classifier__solver'],
                                        max_iter = 5000)
    accuracy_model.fit(X_train, y_train)
    y_acc_pred = accuracy_model.predict(X_train)
    acc_train_score = accuracy_score(y_train, y_acc_pred)
    
    # PRECISION
    solution['Precision'] = hyperparams.cv_results_['mean_test_precision']
    best_precision = results[np.argmin(hyperparams.cv_results_['rank_test_precision'])]
    precision_model = LogisticRegression(penalty = best_precision['classifier__penalty'],
                                         C = best_precision['classifier__C'],
                                         solver = best_precision['classifier__solver'],
                                         max_iter = 5000)
    precision_model.fit(X_train, y_train)
    y_prec_pred = precision_model.predict(X_train)
    prec_train_score = precision_score(y_train, y_prec_pred)
    
    # ROC AUC
    solution['ROC AUC'] = hyperparams.cv_results_['mean_test_roc_auc']
    best_roc_auc = results[np.argmin(hyperparams.cv_results_['rank_test_roc_auc'])]
    roc_model = LogisticRegression(penalty = best_roc_auc['classifier__penalty'],
                                   C = best_roc_auc['classifier__C'],
                                   solver = best_roc_auc['classifier__solver'],
                                   max_iter = 5000)
    roc_model.fit(X_train, y_train)
    y_roc_pred = roc_model.predict(X_train)
    roc_train_score = roc_auc_score(y_train, y_roc_pred)
    
    # F1
    solution['F1'] = hyperparams.cv_results_['mean_test_f1']
    best_f1 = results[np.argmin(hyperparams.cv_results_['rank_test_f1'])]
    f1_model = LogisticRegression(penalty = best_f1['classifier__penalty'],
                                  C = best_f1['classifier__C'],
                                  solver = best_f1['classifier__solver'],
                                  max_iter = 5000)
    f1_model.fit(X_train, y_train)
    y_f1_pred = f1_model.predict(X_train)
    f1_train_score = f1_score(y_train, y_f1_pred)
    
    train_metrics = train_metrics.append({'Accuracy': acc_train_score, 'Precision': prec_train_score, 
                                          'AUC': roc_train_score, 'F1': f1_train_score}, ignore_index=True)
train_metrics

/Users/shaniaie/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shaniaie/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shaniaie/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shaniaie/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics

/Users/shaniaie/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/shaniaie/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/shaniaie/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shaniaie/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter

/Users/shaniaie/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shaniaie/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shaniaie/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shaniaie/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics

/Users/shaniaie/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shaniaie/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shaniaie/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shaniaie/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics

,Accuracy,Precision,AUC,F1
0,0.89,0.740741,0.638957,0.677966
